In [1]:
from pylab import *
from crpropa import *

# Synthetic turbulent field with CRPropa

## Uniform field $\vec{B_0}=B_0\hat{z}$ with turbulence $\vec{\delta B}=(\delta B_x,\delta B_y,0)$

## $\delta B_x=\delta B\cos(kz)$ and $\delta B_y=\pm\delta B\sin(kz)$ such that $\frac{\delta B_y}{\delta B_x}=\pm\tan(kz)$

In [ ]:
random_seed = 42
Brms = 3*nG
lmin = 60*pc
lmax = 800*pc
index = 5./3.
spectrum = SimpleTurbulenceSpectrum(Brms, lmin, lmax, index)
grid = GridProperties(Vector3d(0), 256, 30*pc)
BField = SimpleGridTurbulence(spectrum, grid, random_seed)

In [ ]:
print('Lc = {:.1f} kpc'.format(BField.getCorrelationLength() / kpc))  
print('sqrt(<B^2>) = {:.1f} nG'.format(BField.getBrms() / nG))   
print('<|B|> = {:.1f} nG'.format(BField.getMeanFieldStrength() / nG))

In [ ]:
BField = dumpGrid(BField.getGrid(), 'myfield.dat')


for i in range(0,601):
    z = 10**(i / 100)
    Bx, By, Bz = BBField.getField(Vector3d(0,0,z) * Mpc) / nG
    k = (By / Bx)*(1 / z) 
    delta_B = np.cos(k*z) / Bx
    F_k = delta_B**2 * k
    
    
plt.plot(k,)